# Setup

In [36]:
%cd  /home/tlm/Work/FYP-22-23/weee-preprocess

/home/tlm/Work/FYP-22-23/weee-preprocess


In [37]:
import os
import datetime
import numpy as np
import pandas as pd

In [38]:
DATASET_DIR = "./data/v1"

# Prepare Data

In [39]:
from utils.dataset import StudyInfoEncoder, DatasetVersion1
from utils.naming import standardize_column_names

In [40]:
dataset = DatasetVersion1()
left, right = dataset.earbud_acc(participant_id=1)

In [46]:
df = left
df['timestamp'].iloc[-1]

Timestamp('2021-12-03 17:29:13.382000128+0000', tz='UTC')

In [54]:
df = pd.concat([
    left, 
    pd.DataFrame({
        'timestamp': pd.date_range(df['timestamp'].iloc[0], df['timestamp'].iloc[-1], freq='100ms'),
        'ax': np.nan, 'ay': np.nan, 'az': np.nan
    })
], axis=0).set_index('timestamp').sort_index()

In [56]:
df.rolling('30s', center=True, min_periods=30*10).mean()

,ax,ay,az
timestamp,,,
2021-12-03 16:53:50+00:00,16381.677852,443.701342,1835.374161
2021-12-03 16:53:50+00:00,16381.677852,443.701342,1835.374161
2021-12-03 16:53:50.100000+00:00,16386.937397,452.683690,1842.568369
2021-12-03 16:53:50.200000+00:00,16380.110211,461.625608,1843.377634
2021-12-03 16:53:50.300000+00:00,16387.030255,457.649682,1847.789809
...,...,...,...
2021-12-03 17:29:13.343000064+00:00,-2295.493937,7674.203574,-13798.239311
2021-12-03 17:29:13.352999936+00:00,-2296.049170,7674.978927,-13797.556833
2021-12-03 17:29:13.361999872+00:00,-2296.599361,7675.780831,-13796.892652
